<a href="https://colab.research.google.com/github/AmineSdk/RecommenderSystem/blob/main/Social_MoleTrust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [ ]:
! kaggle datasets download -d aminesedkaoui/yelp_review_ratings
! kaggle datasets download -d aminesedkaoui/yelp_user_friends

In [ ]:
#constructing trust_dataset according to usersList minimization.
def dataset_trust_construct(user,friends,usersList,data_trustdatasetList2DF) :
  if user in usersList :
    friends = friends.split(", ")
    # print(friends)
    # print(len(friends))
    for friend in friends :
      # print('in for')
      # print(friend)
      if friend in usersList :
        # print('in if')
        # print(friend)
        row = []
        row.append(user)
        row.append(friend)
        row.append(1)
        # print(row)
        data_trustdatasetList2DF.append(row)
        print(len(data_trustdatasetList2DF))
  return data_trustdatasetList2DF

def check (user1 , user2 , trustval, usersList, df_user_user) :
  if (user1 in usersList) and (user2 in usersList) :
    df_user_user.at[user1, user2] = trustval
  return df_user_user


def fill_user_matrix (usersList, df_user_user, dataset_trust) : 
  #df_user_user = dataset_trust.apply(lambda row : check(row['userID1'], row['userID2'], row['trustValue'], usersList, df_user_user), axis = 1)
  dataset_trust.apply(lambda row : check(row['userID1'], row['userID2'], row['trustValue'], usersList, df_user_user), axis = 1)
  return df_user_user
  

def moleTrustAlgorithme (dataset_rating , dataset_trust, usersList) : 

    usersNumber = len(usersList) 
    matrix_user_user = np.zeros((usersNumber, usersNumber))

    df_user_user = pd.DataFrame(matrix_user_user, index = dataset_rating['userID'].unique()  ,columns = dataset_rating['userID'].unique()) 
    df_user_user = fill_user_matrix (usersList, df_user_user, dataset_trust)
    print("df_user_user trust values after-filling :")
    print(df_user_user)



    g = Graph() #directed
    #maybe better. . > dictionnary construction < . .###################################################### 
    val2id = {}
    for user in dataset_rating['userID'].unique() :
      #add node to g
      v = g.add_vertex() 
      #add it index to its user in the dictionarry
      val2id[g.vertex_index[v]] = user

    ##########################################################################

    for i in range(usersNumber):
      for j in range(usersNumber):
        s1 = val2id[i] #users ID
        s2 = val2id[j]

        if df_user_user.at[s1, s2] != 0:
          g.add_edge(g.vertex(i), g.vertex(j))


    for a in range(usersNumber):
      for b in range(usersNumber):

        aUser = val2id[a] #users ID
        bUser = val2id[b]

        if df_user_user.at[aUser, bUser] == 0 and a != b:
          Val_1 = 0
          Val_2 = 0
          Trust_propagation = 0

          for path in graph_tool.topology.all_paths(g, g.vertex(a), g.vertex(b), cutoff=4, edges=True) :
            W_k = 1
            W_direct = 1
            edge = len(path)
            cpt = 0

            for current_edge in path:
              cpt = cpt + 1
              #get vertexs
              s1 = current_edge.source()
              s2 = current_edge.target()
              #get index of vertexs
              index1 = g.vertex_index[s1]
              index2 = g.vertex_index[s2]
              #get Users of indexes
              s1User = val2id[index1]
              s2User = val2id[index2]
              #get trust between the 2 users
              Trust_value = df_user_user.at[s1User, s2User]
              # # # # # # # # # # # #
              if cpt < edge :
                W_k = W_k * Trust_value
              if cpt == edge :
                W_direct = Trust_value

            Val_1 = Val_1 + W_direct * W_k
            Val_2 = Val_2 + W_k


          if Val_1 == 0 and Val_2 == 0:
                    Trust_propagation = 0
          else:
                    Trust_propagation = Val_1 / Val_2
                
          #Set Trust
          df_user_user.at[aUser, bUser] = Trust_propagation
    
    return df_user_user


In [ ]:
#ratings
dataset_rating = pd.read_csv("/content/yelp_review_ratings.csv")
dataset_rating.rename(columns = {'text': 'reviewText', 'stars': 'rating', 'business_id': 'productID', 'user_id': 'userID'}, inplace = True)
#dataset_rating = dataset_rating[:100]
usersList = dataset_rating['userID'].unique()
print(len(usersList))

print(len(dataset_rating))
########################
#trusts
userfriends = pd.read_csv("/content/yelp_user_friends.csv")
userfriends.rename(columns = {'user_id': 'userID'}, inplace = True)
print(userfriends.head(5))
data_trustdatasetList2DF_cols = ['userID1','userID2','trustValue']
data_trustdatasetList2DF = []
userfriends.apply(lambda row : dataset_trust_construct(row['userID'], row['friends'], usersList, data_trustdatasetList2DF), axis = 1)

print(len(data_trustdatasetList2DF))
if len(data_trustdatasetList2DF) != 0 :
  dataset_trust = pd.DataFrame(data_trustdatasetList2DF, columns=data_trustdatasetList2DF_cols)
else :
  print("Error trust dataset not filled.")
  dataset_trust = 0

print(dataset_trust.head(5))
dataset_trust.to_csv('/content/yelp_user_user_trust.csv')